In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

In [ ]:
!pip install tiffile
!pip install elasticdeform
!pip install keras==2.2.5
!pip install csbdeep
!pip install stardist

In [ ]:
%cd '/content/drive/My Drive/BTrack/SegmentationTerminator/Terminator/'
import sys
import os

import cv2
import numpy as np
from skimage.util import invert
%matplotlib inline
from skimage import measure
%config InlineBackend.figure_format = 'retina'
sys.path.append('../../../Terminator/')
import glob
from csbdeep.utils import Path, normalize
from tifffile import imread
from stardist.models import StarDist2D
from TerminatorUtils.helpers import normalizeFloat, OtsuThreshold2D, save_8bit_tiff_imagej_compatible,Integer_to_border, RelabelZ
from TerminatorUtils.helpers import Prob_to_Binary, zero_pad, multiplot, CCLabels,doubleplot, remove_big_objects,fill_label_holes
from TerminatorUtils.helpers import save_tiff_imagej_compatible, SeedStarDistWatershedAll,MaxProjectDist,SeedStarDistWatershedV2
from csbdeep.models import Config, CARE

from skimage.filters import threshold_local, threshold_mean, threshold_otsu
import pandas as pd
from pandas import DataFrame, Series  # for convenience


In [ ]:
basedir = '/content/drive/My Drive/BTrack/Claudia_Green/'
savedir = basedir + 'Denoised/'
savesegmenteddir = savedir + '_Segmented/'
Model_Dir = '/content/drive/My Drive/BTrack/SegmentationModel/'

NoiseModelName = '3DDenoisingModel'
UNETSegmentationModelName = 'FakeMembraneMask'
StardistModelName = 'FakeMembraneSmartSeeds'


max_size = 5000
threshold = 50
denoise = True

NoiseModel = CARE(config = None, name = NoiseModelName, basedir = Model_Dir)
model = StarDist2D(config = None, name = StardistModelName, basedir = Model_Dir)
UnetModel = CARE(config = None, name = UNETSegmentationModelName, basedir = Model_Dir)
Path(savedir).mkdir(exist_ok = True)
Path(savesegmenteddir).mkdir(exist_ok = True)

In [ ]:
Raw_path = os.path.join(basedir, '*tif')
axes = 'ZYX'

if denoise:
  filesRaw = glob.glob(Raw_path)
  for fname in filesRaw:
    
          Greenimage = imread(fname)
          
          
          Name = os.path.basename(os.path.splitext(fname)[0])
          print('Denoising Image')
          Greenimage = NoiseModel.predict(Greenimage,axes, n_tiles = (1,2,2))
             
          save_tiff_imagej_compatible((savedir  + Name+ '.tif' ) , Greenimage, axes)
          print('Denoised Image saved')

                

In [ ]:
Raw_path = os.path.join(savedir, '*tif') 
 
min_size = 10 
axis_norm = (0,1)
axes = 'ZYX'
saveaxes = 'ZYX'

filesRaw = glob.glob(Raw_path) 
for fname in filesRaw:

     print(fname) 
     #Read image        
      image = imread(fname)
 
      originalX = image.shape[1]
      originalY = image.shape[2]  

 
      Name = os.path.basename(os.path.splitext(fname)[0])
      #Declare bunch of files  
      StarsegmentationImage = np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype = ('uint16'))
      prob = np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype = ('uint16'))
      Watershed = np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype = ('uint16'))
      Volume =  np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype = ('uint16')) 
      Markers = np.zeros([image.shape[0], image.shape[1], image.shape[2]], dtype = ('uint16')) 
      Finalimage = np.zeros([image.shape[0],image.shape[1],image.shape[2]], dtype = ('uint16'))
      Segmented= np.zeros([image.shape[0],image.shape[1],image.shape[2]], dtype = ('uint16'))
      #Loop over Z  
      for i in range(0, image.shape[0]):
        x = image[i,:]
 
 
        #Make sure image is 2D
      
        
        Segmented[i,:] = UnetModel.predict(x,axes, n_tiles=(8,8))
        thresh = threshold_otsu(Segmented[i,:])
        Finalimage[i,:] = Segmented[i,:] > thresh 
        


        x = normalize(x,1,99.8,axis=axis_norm)
        
        x = zero_pad(x, 64, 64)
        
       
        #Get stardist, label image, details, probability map, distance map
        MidImage, details = model.predict_instances(x, n_tiles=(8,8))
        StarsegmentationImage[i,:] = MidImage[:originalX, :originalY]
        smallprob, smalldist = model.predict(x, n_tiles=(8,8))
        grid = model.config.grid
        midprob = cv2.resize(smallprob, dsize=(smallprob.shape[1] * grid[1] , smallprob.shape[0] * grid[0] ))
        middist = cv2.resize(smalldist, dsize=(smalldist.shape[1] * grid[1] , smalldist.shape[0] * grid[0] ))
        dist = MaxProjectDist(middist)
        prob[i,:] = dist[:originalX, :originalY] 
 

      
 
        #Seeds from Stardist, segmentation on probability map
        Watershed[i,:], Markers[i,:] = SeedStarDistWatershedClaudia(prob[i,:],StarsegmentationImage[i,:].astype('uint16'),Finalimage[i,:],  model.config.grid,max_size = max_size,  min_size = min_size)    
       
        Watershed[i,:] = fill_label_holes(Watershed[i,:].astype('uint16'))
        Watershed[i,:] = remove_big_objects(Watershed[i,:], max_size = max_size)


        properties = measure.regionprops(Watershed[i,:].astype('uint16'), image[i,:])
        Labelindex = [prop.label for prop in properties] 
        if i%2 == 0:
          print('Zpoint', i)
          print('Total number of Tomatos at Zpoint', i, 'is', len(Labelindex))
          multiplot(Finalimage[i,:],prob[i,:],Markers[i,:], 'Mask', 'Distance Map', 'Markers', plotTitle = 'Segmentation Input' )
          multiplot(x,Watershed[i,:],StarsegmentationImage[i,:].astype('uint16'), 'Original',  'SmartSeeds', 'StarDist', plotTitle = 'Super Segmentation' )
 
      
      Volume = merge_labels_across_volume(Watershed.astype('uint16'), RelabelZ, threshold)
      #Save best segmentation
    
      save_tiff_imagej_compatible((savesegmenteddir+ 'Volume'  + Name+ '.tif' ) , Volume, saveaxes)

/Users/aimachine/miniconda3/envs/tensorflowEnv36/lib/python3.6/site-packages/scipy/ndimage/measurements.py:1359: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]
[INFO][2020/06/05 12:35:47 PM] btrack (v0.3.8) library imported
[INFO][2020/06/05 12:35:47 PM] Starting BayesianTracker session
[INFO][2020/06/05 12:35:47 PM] Loading configuration file: ./cell_config.json
[INFO][2020/06/05 12:35:47 PM] Loading motion model: b'cell_motion'
[INFO][2020/06/05 12:35:47 PM] Set volume to ((0, 744), (0, 700), (-100000.0, 100000.0))
[INFO][2020/06/05 12:35:47 PM] Starting tracking... 
[INFO][2020/06/05 12:35:48 PM] Tracking objects in frames 0 to 4(of 4)...
[INFO][2020/06/05 12:35:49 PM]  - Timing (Bayesian updates: 519.01ms, Linking: 7.36ms)
[INFO][2020/06/05 12:35:49 PM]  - Probabilities (Link: 0.99994, Lost: 0.76746)
[INFO][2020/06/05 12:35:49 PM] SUCCESS.
[INFO][2020/06/05 12:35:49 PM]  - Found 1537 tracks in 4 frames (in 0.0s)
[INFO][2020/06/05 12:35:49 P